In [2]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('../cleaned_data.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851422 entries, 0 to 1851421
Data columns (total 36 columns):
 #   Column             Dtype  
---  ------             -----  
 0   FlightDate         object 
 1   Quarter            int64  
 2   Year               int64  
 3   Month              int64  
 4   DayofMonth         int64  
 5   DepTime            float64
 6   DepDel15           float64
 7   CRSDepTime         int64  
 8   DepDelayMinutes    float64
 9   OriginAirportID    int64  
 10  DestAirportID      int64  
 11  ArrTime            float64
 12  CRSArrTime         int64  
 13  ArrDelayMinutes    float64
 14  Origin             object 
 15  Dest               object 
 16  RoundedFlightDate  object 
 17  DepatHr            int64  
 18  ArrDel15           float64
 19  Delayed            int64  
 20  date               object 
 21  airport            object 
 22  windspeedKmph      int64  
 23  winddirDegree      int64  
 24  weatherCode        int64  
 25  precipMM          

In [6]:
data = pd.concat([data, pd.get_dummies(data['Origin'],prefix='Origin')], axis=1)

In [7]:
data = pd.concat([data, pd.get_dummies(data['Dest'],prefix='Destination')], axis=1)

In [8]:
data.columns

Index(['FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime',
       'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'OriginAirportID',
       'DestAirportID', 'ArrTime', 'CRSArrTime', 'ArrDelayMinutes', 'Origin',
       'Dest', 'RoundedFlightDate', 'DepatHr', 'ArrDel15', 'Delayed', 'date',
       'airport', 'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'WindChillF', 'humidity', 'time', 'timeInHr', 'Origin_ATL',
       'Origin_CLT', 'Origin_DEN', 'Origin_DFW', 'Origin_EWR', 'Origin_IAH',
       'Origin_JFK', 'Origin_LAS', 'Origin_LAX', 'Origin_MCO', 'Origin_MIA',
       'Origin_ORD', 'Origin_PHX', 'Origin_SEA', 'Origin_SFO',
       'Destination_ATL', 'Destination_CLT', 'Destination_DEN',
       'Destination_DFW', 'Destination_EWR', 'Destination_IAH',
       'Destination_JFK', 'Destination_LAS', 'Destination_LAX',
       'Destination_MCO', 'Destination_MIA', 'Destination_O

In [175]:
Df = data[[ 'Quarter', 'Month', 'DayofMonth',
       'CRSDepTime', 'CRSArrTime', 
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'WindChillF', 'humidity',  'Origin_ATL',
       'Origin_CLT', 'Origin_DEN', 'Origin_DFW', 'Origin_EWR', 'Origin_IAH',
       'Origin_JFK', 'Origin_LAS', 'Origin_LAX', 'Origin_MCO', 'Origin_MIA',
       'Origin_ORD', 'Origin_PHX', 'Origin_SEA', 'Origin_SFO',
       'Destination_ATL', 'Destination_CLT', 'Destination_DEN',
       'Destination_DFW', 'Destination_EWR', 'Destination_IAH',
       'Destination_JFK', 'Destination_LAS', 'Destination_LAX',
       'Destination_MCO', 'Destination_MIA', 'Destination_ORD',
       'Destination_PHX', 'Destination_SEA', 'Destination_SFO',  'DepDel15']]


In [176]:
Df.head()

,Quarter,Month,DayofMonth,CRSDepTime,CRSArrTime,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,...,Destination_JFK,Destination_LAS,Destination_LAX,Destination_MCO,Destination_MIA,Destination_ORD,Destination_PHX,Destination_SEA,Destination_SFO,DepDel15
0,1,1,1,2347,714,15,123,113,0.0,10,...,0,0,0,0,1,0,0,0,0,1.0
1,1,1,1,20,705,15,123,113,0.0,10,...,0,0,0,0,0,0,0,0,0,0.0
2,1,1,1,2359,603,15,38,113,0.0,10,...,0,0,0,0,0,1,0,0,0,0.0
3,1,1,1,30,823,15,38,113,0.0,10,...,0,0,0,0,1,0,0,0,0,0.0
4,1,1,1,10,501,15,38,113,0.0,10,...,0,0,0,0,0,0,0,0,0,0.0


In [327]:
X_train, X_test, y_train, y_test = train_test_split(Df.iloc[:, :-1], Df.iloc[:, -1], train_size=0.75, stratify=Df.iloc[:, -1].values, random_state=0)



In [328]:
X_train.shape, y_train.shape

((1388566, 47), (1388566,))

In [329]:
X_test.shape, y_test.shape

((462856, 47), (462856,))

In [330]:
y_train.value_counts()

0.0    1109510
1.0     279056
Name: DepDel15, dtype: int64

In [331]:
# Negative to positive ratio
ratio = 1109510/279056
ratio

3.9759403130554443

In [332]:
# xgb = XGBClassifier( n_estimators=400, colsample_bytree= 0.8, gamma= 0,learning_rate= 0.3, max_depth= 10,reg_lambda= 15,scale_pos_weight= 3.77, subsample= 0.8)
# xgb = XGBClassifier()
xgb = XGBClassifier( n_estimators=400, gamma= 0,learning_rate= 0.7, max_depth= 20,scale_pos_weight= 11)


In [333]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=23)
X_train, y_train = ros.fit_resample(X_train.values, y_train.values)

In [334]:
np.bincount(y_train.astype('int'))

array([279056, 279056])

In [335]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_train[0]

array([-1.37386354, -1.33726976, -0.65273038,  1.50705574, -2.79210543,
        0.04737258,  1.08104368, -0.49764994, -0.28074459,  0.32267497,
        1.12150771, -0.91509815, -2.13728403,  0.08962505, -2.10959186,
       -2.03429307, -0.7762511 ,  3.47757626, -0.21311464, -0.28770265,
       -0.26853477, -0.24473009, -0.22270971, -0.23096664, -0.27454096,
       -0.36785878, -0.23695185, -0.20941258, -0.30690961, -0.24808445,
       -0.24961903, -0.32379977, -0.28489299, -0.20558898, -0.28387352,
       -0.26454799, -0.2533923 , -0.21847132, -0.2404822 , -0.27753683,
        2.76087305, -0.23895685, -0.20574988, -0.29492713, -0.25303173,
       -0.25412026, -0.33491645])

In [336]:
%%time
model = xgb.fit(X_train ,y_train)
# model = xgb.fit(X_train.values ,y_train.values)

CPU times: user 43min 14s, sys: 7min 32s, total: 50min 47s
Wall time: 7min 33s


In [337]:
X_test = sc.transform(X_test)
X_test[0]

array([ 1.37697528,  1.04584227,  1.28873002, -1.0687852 ,  0.13515864,
       -0.86200897, -1.9937911 , -0.49764994, -0.28074459,  0.32267497,
        0.3362275 , -1.0916482 , -0.91005133, -0.77643502, -0.32818393,
       -0.27724853, -1.08912787, -0.2875566 , -0.21311464, -0.28770265,
       -0.26853477, -0.24473009, -0.22270971, -0.23096664,  3.6424437 ,
       -0.36785878, -0.23695185, -0.20941258, -0.30690961, -0.24808445,
       -0.24961903, -0.32379977, -0.28489299, -0.20558898, -0.28387352,
       -0.26454799, -0.2533923 , -0.21847132, -0.2404822 , -0.27753683,
       -0.36220427,  4.18485598, -0.20574988, -0.29492713, -0.25303173,
       -0.25412026, -0.33491645])

In [338]:
prediction = model.predict(X_test)

In [339]:
# predic_prob = np.max(model.predict_proba(X_test), axis=1)

In [340]:
# predic_prob

In [341]:
# predictions_2 = np.where(predic_prob >= 0.999, 1, 0)

In [342]:
# predictions_2[:100]

In [312]:
y_test.values[:100]

array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [343]:
accuracy_score(prediction, y_test.values )

0.6481821560053235

In [344]:
f1_score(prediction, y_test.values )

0.4645765860555345

In [345]:
print(classification_report(prediction, y_test))

              precision    recall  f1-score   support

           0       0.62      0.91      0.74    251740
           1       0.76      0.33      0.46    211116

    accuracy                           0.65    462856
   macro avg       0.69      0.62      0.60    462856
weighted avg       0.68      0.65      0.61    462856

